# 📌 Uygulama – 5  
# 🧠 “LibTorch + OpenCV: Modüler Preprocess & Inference Pipeline”  

Bu uygulama, önceki tüm uygulamaların birleşip **gerçek bir C++ inference mimarisine** dönüştüğü aşamadır.  
Kod artık fonksiyonlara ayrılmıştır ve bir üretim (production) pipeline’ına benzemektedir.

Bu uygulamada:

- Preprocess ayrı fonksiyon  
- Inference ayrı fonksiyon  
- Hata kontrol mekanizması yerinde  
- Shape doğrulama yapılıyor  
- Tensor memory handling doğru  
- Çıkış softmax + argmax ile sınıf tahmini  

Yani ilk kez **tam bir C++ derin öğrenme inference pipeline’ı kurdun.**

---

## 🎯 1) Amaç — *Bu projede ne yaptım?*

Bu uygulamanın amacı şuydu:

> **Python’da eğittiğim CNN modelini, LibTorch + OpenCV ile modüler bir C++ pipeline halinde çalıştırmak:  
> preprocess → tensor → forward → postprocess.**

Yani tek görüntü üzerinde inference ama **profesyonel standartta**.

---

## 🧩 2) Ana Adımlar — *Kodda yaptığım işlemler*

---

### 🟦 A) `preprocess_image()` fonksiyonu  
Bu fonksiyon, bir görüntüyü modele uygun hale getiriyor.

#### 1️⃣ Resize (model input boyutuna)
```cpp
cv::resize(img, img, cv::Size(target_size, target_size));
```


#### 2️⃣ BGR → RGB
```cpp
cv::cvtColor(img, img, cv::COLOR_BGR2RGB);
```

#### 3️⃣ uint8 → float32 + normalize
```cpp
img.convertTo(img_float, CV_32F, 1.0f / 255.0f);
```

#### 4️⃣ HWC → NHWC tensör oluşturma
```cpp
auto tensor_image = torch::from_blob(
    img_float.data,
    {1, img_float.rows, img_float.cols, 3},
    torch::TensorOptions().dtype(torch::kFloat32)
);
```

#### 5️⃣ NHWC → NCHW dönüşümü + clone()
```cpp
tensor_image = tensor_image.permute({0,3,1,2});
tensor_image = tensor_image.contiguous().clone();
```


Bu adım ile tensör artık:

* [1, 3, 64, 64] formatında

*  LibTorch’un kendi belleğinde

* Modele direkt verilmeye hazır


### 🟩 B) run_inference() fonksiyonu

Modelin forward işlemini yapan modüler fonksiyon:
```cpp
std::vector<torch::jit::IValue> inputs;
inputs.push_back(input);

torch::Tensor output = module.forward(inputs).toTensor();
```


Bu yapı:

* Üretim (production) inference pipeline’larında kullanılan standarttır.

* Model ile preprocess ayrılmıştır.

* Unit-test yapılabilir hale gelir.


### 🟥 C) Ana program akışı
✔ 1) Model dosyasını doğruladım

* Exe ile aynı klasörde mi?

* Değilse kullanıcıya açıklayıcı hata sundum

✔ 2) TorchScript modeli yükledim
```cpp
module = torch::jit::load(model_path);
module.to(torch::kCPU);
module.eval();
```

✔ 3) test.jpg dosyasını yükledim ve doğruladım

✔ 4) Preprocess → preprocess_image(img_bgr, 64)

✔ 5) Tensor shape doğrulaması yaptım

Bu, profesyonel seviyede çok önemli bir güvenlik adımı:
```cpp
if (!(sizes.size()==4 && sizes[0]==1 && sizes[1]==3 ...)) { ... }
```

✔ 6) Inference çalıştırdım
```cpp
torch::Tensor output = run_inference(module, input_tensor);
```

✔ 7) Softmax + Argmax postprocess
```cpp
torch::softmax(output, 1);
probs.argmax(1);
```


✔ 8) Logits ve tahmin edilen sınıfı ekrana yazdırdım

---

## 🧠 3) Bu projede öğrendiğim temel kavramlar

📌 Preprocess ve inference fonksiyonlarını modüler ayırma

📌 Üretim pipeline mimarisi kurma

📌 LibTorch + OpenCV birlikte çalışma modeli

📌 Hata kontrolü → std::ifstream, c10::Error

📌 Tensor shape validation (çok önemli)

📌 contiguous().clone() ile bellek yönetimi

📌 Vector<IValue> ile forward input hazırlama

📌 Softmax & Argmax ile sınıf tahmini

📌 Pipeline mantığı:
**Load Model → Load Image → Preprocess → Forward → Postprocess**

---

## ⚠️ 4) Dikkat edilmesi gereken noktalar

model_ts.pt ve test.jpg mutlaka EXE ile aynı klasörde

Input shape sabit olmalı → 64×64

BGR → RGB yapılmazsa model saçmalar

Normalize yapılmazsa modelin accuracy’si çöker

NHWC → NCHW dönüşümü yapılmazsa runtime error alırsın

from_blob mutlaka clone() ile ayrılmalı

Forward’dan önce model eval() modunda olmalı

---

## 🔖 5) Etiketler / Kategori

Advanced Preprocess

Inference Pipeline

TorchScript + OpenCV

CNN Classifier

Softmax + Argmax

Production Style C++ Code

--- 

## 📝 6) Tek Cümlelik Özet

TorchScript CNN modelimi C++ tarafında profesyonel bir preprocess + inference pipeline’ına dönüştürdüm ve tek bir görüntüde doğru sınıf tahmini aldım.